In [4]:
from langchain.llms import LlamaCpp
from langchain.embeddings import LlamaCppEmbeddings

local_path = "../../models/game_npc_vicuna_huntress/ggml-f16.bin"
llm = LlamaCpp(model_path=local_path, verbose=True, n_batch=256, temperature=0.3, n_ctx=2048, use_mmap=False, stop=["###"])


llama.cpp: loading model from ../../models/game_npc_vicuna_huntress/ggml-f16.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 49954
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size = 13448820.20 KB
llama_model_load_internal: mem required  = 14925.61 MB (+ 1026.00 MB per state)
....................................................................................................
llama_init_from_file: kv self size  = 1024.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 1 | AVX512_VNNI = 1 | FMA = 1 

In [9]:
from retrivers.llama_time_weighted_retriever import LlamaTimeWeightedVectorStoreRetriever
from vectorestores.chroma import EnhancedChroma

def create_new_memory_retriever():
    embeddings_model = LlamaCppEmbeddings(model_path=local_path)
    vs = EnhancedChroma(embedding_function=embeddings_model)
    return LlamaTimeWeightedVectorStoreRetriever(vectorstore=vs, other_score_keys=["importance"], k=14)


In [10]:
from generative_agents.llama_generative_agent import LlamaGenerativeAgent
from generative_agents.llama_memory import LlamaGenerativeAgentMemory

tommies_memory = LlamaGenerativeAgentMemory(
    llm=llm,
    memory_retriever=create_new_memory_retriever(),
    reflection_threshold=8, # we will give this a relatively low number to show how reflection works
    verbose=True,
)

llama.cpp: loading model from ../../models/game_npc_vicuna_huntress/ggml-f16.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 49954
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  68.20 KB
llama_model_load_internal: mem required  = 14925.61 MB (+ 2052.00 MB per state)
llama_init_from_file: kv self size  =  512.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 1 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
Using 

In [11]:
tommie = LlamaGenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative", # You can add more persistent traits here
    status="looking for a job", # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=llm,
    memory=tommies_memory,
    verbose=True,
)

llama.cpp: loading model from ../../models/game_npc_vicuna_huntress/ggml-f16.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 49954
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  68.20 KB
llama_model_load_internal: mem required  = 14925.61 MB (+ 2052.00 MB per state)
llama_init_from_file: kv self size  =  512.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 1 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
Using 

In [12]:
print(tommie.get_summary(force_refresh=True))


llama_print_timings:        load time =   805.15 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =   804.37 ms /     8 tokens (  100.55 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =   807.88 ms




> Entering new LLMChain chain...
Prompt after formatting:
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
###USER: Summarize Tommie's core characteristics given the following input. Do not embellish if you don't know. Do not return a list.
Input: 

###ASSISTANT: 

> Finished chain.
Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is an optimistic, outgoing person who loves to explore new places and try different foods. She enjoys spending time with her friends and family, and she has a passion for photography.


In [13]:
# We can add memories directly to the memory object
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]
for observation in tommie_observations:
    tommie.memory.add_memory(observation)



> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: I met my wife Jane when I was 23
Rating: 9
Memory: I visited Italy in 2020
Rating: 5
Memory: Tommie remembers his dog, Bruno, from when he was a kid
Rating: 


Llama.generate: prefix-match hit
Importance score: -10



> Finished chain.


ValueError: could not convert string to float: '-'

In [ ]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

In [6]:
print(tommie.generate_reaction("Tommie sees his neighbor's cat"))



> Entering new LLMChain chain...
Prompt after formatting:
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
###USER: Extract the entity from the following observation without explanation.
Observation: Tommie sees his neighbor's cat

###ASSISTANT: 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
###USER: What is the ​ doing in the following observation?
Observation: Tommie sees his neighbor's cat

###ASSISTANT: 

> Finished chain.



llama_print_timings:        load time =  4570.63 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  8955.84 ms /    12 tokens (  746.32 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  8956.70 ms
Chroma collection langchain contains fewer than 100 elements.
Chroma collection langchain contains fewer than 99 elements.
Chroma collection langchain contains fewer than 98 elements.
Chroma collection langchain contains fewer than 97 elements.
Chroma collection langchain contains fewer than 96 elements.
Chroma collection langchain contains fewer than 95 elements.
Chroma collection langchain contains fewer than 94 elements.
Chroma collection langchain contains fewer than 93 elements.
Chroma collection langchain contains fewer than 92 elements.
Chroma collection langchain contains fewer than 91 elements.
Chroma collecti



> Entering new LLMChain chain...
Prompt after formatting:
What is the relationship between Tommie and ​?
Context from memory:
- May 05, 2023, 11:24 PM: Tommie remembers his dog, Bruno, from when he was a kid
- May 05, 2023, 11:25 PM: Tommie sees the new home
- May 05, 2023, 11:25 PM: The road is noisy at night
- May 05, 2023, 11:25 PM: The new neighbors have a cat
- May 05, 2023, 11:39 PM: Tommie observed Tommie sees a cat and reacted by Tommie's reaction: Tommie might be curious about the cat and ask where it came from, or he might simply acknowledge its presence without saying anything.
- May 05, 2023, 11:25 PM: Tommie is hungry
- May 05, 2023, 11:24 PM: Tommie feels tired from driving so far
- May 05, 2023, 11:25 PM: Tommie tries to get some rest.
Relevant context:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
1. Tommie remembers his dog Bruno from when he was a kid.
2. Tommie s

Importance score: 8



> Finished chain.
(False, "Tommie's reaction: Tommie might be curious about the cat and ask where it came from, or he might simply acknowledge its presence without saying anything.")



llama_print_timings:        load time =  4570.63 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 12674.51 ms /    53 tokens (  239.14 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time = 12675.86 ms


In [7]:
tommie.generate_dialogue("Dad", "Have you got a new job?")


llama_print_timings:        load time =  4570.63 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  5686.36 ms /     3 tokens ( 1895.45 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  5687.03 ms
Chroma collection langchain contains fewer than 100 elements.
Chroma collection langchain contains fewer than 99 elements.
Chroma collection langchain contains fewer than 98 elements.
Chroma collection langchain contains fewer than 97 elements.
Chroma collection langchain contains fewer than 96 elements.
Chroma collection langchain contains fewer than 95 elements.
Chroma collection langchain contains fewer than 94 elements.
Chroma collection langchain contains fewer than 93 elements.
Chroma collection langchain contains fewer than 92 elements.
Chroma collection langchain contains fewer than 91 elements.
Chroma collecti



> Entering new LLMChain chain...
Prompt after formatting:
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
###USER: what is the most possible relationship between Tommie and Dad in the following observation? Do not embellish if you don't know. Do not return a list.
Observation: - May 05, 2023, 11:44 PM: Tommie observed Tommie sees his neighbor's cat and reacted by Tommie's reaction: Tommie might be curious about the cat and ask where it came from, or he might simply acknowledge its presence without saying anything.
- May 05, 2023, 11:24 PM: Tommie remembers his dog, Bruno, from when he was a kid
- May 05, 2023, 11:25 PM: Tommie sees the new home
- May 05, 2023, 11:25 PM: The road is noisy at night
- May 05, 2023, 11:39 PM: Tommie observed Tommie sees a cat and reacted by Tommie's reaction: Tommie might be curious about the cat and ask where it came from, or he might simply acknow

(False, '1. What to say: "No, I haven\'t found one yet."')